In [ ]:
# imports
from standardization.tokenization import tokenize
from standardization.tagging import tag_tokens, df_tags, remove_perso_info
from utils.csv_io import import_csv, export_csv
from utils.sample import Sample
from HMM.transition import create_train_test_sample, display_statistics, compute_transition_matrix
# from HMM.transition import creckages : on télécharge les données, on installe les librairies qui ne sont pas presentes par défaut et on récupère les fichiers nécessairesate_train_test_sample, compute_transition_matrix
import pandas as pd


In [ ]:
create_sample = False
BUCKET = 'projet-pfe-adress-matching'
FILE_KEY_S3 = 'DonneesCompletes.csv'

if create_sample:
    print("Creating new sample.")
    # import of the data
    full_df = import_csv(BUCKET, FILE_KEY_S3)
    # initialisate a sample
    sample = Sample(dataset=full_df, size=100)
    # create the sample
    sample.create_sample()
    #  put the sample in the BUCKET (avoid to push it by mistake)
    sample.save_sample_file(BUCKET, 'sample.csv')
else:
    print("Importing previously created sample.")
    # import the previous sample
    df_sample = import_csv(BUCKET, 'sample.csv', sep=',')

In [ ]:
df_sample

In [ ]:
# import others datasets
df_sample = import_csv(BUCKET, 'sample.csv', sep=',')
replacement = pd.read_csv('remplacement.csv', sep=",")
lib_voie = pd.read_csv('libvoie.csv', sep=",")

df = df_sample.iloc[:, :8]

# extract addresses column
addresses = df.iloc[:, 0]
cp = df.iloc[:, 1]
communes = df.iloc[:, 2]

In [ ]:
# create tokens for the 100 first addresses
tokens_addresses = tokenize(addresses, replacement_file=replacement)
tokens_communes = tokenize(communes, replacement_file=replacement)
# print(tokens_communes[0:10])
print(len(tokens_communes),len(tokens_addresses))

# tag the tokens with their label
tags = tag_tokens(
    tokens_addresses,
    cp,
    tokens_communes,
    libvoie_file=lib_voie
    )

# remove personal information
tags_without_perso = remove_perso_info(tags)
print(tags_without_perso[0:10])

In [ ]:
train_sample = create_train_test_sample(tags_without_perso)[0]

display_statistics(train_sample)

In [ ]:
transition_matrix = compute_transition_matrix(tags_without_perso)
transition_matrix

In [ ]:
from seaborn import heatmap
heatmap(transition_matrix, cmap="Blues")